Credits for the Spotify related python code goes to the developer website for Spotify, as well as [this blog post](https://dev.to/ericbonfadini/finding-my-new-favorite-song-on-spotify-4lgc) for the general idea.

I will use this code to get a playlist ID of my "metal" playlist, and from there, grab the track by track audio data. Then, I will compare that data to 5,000 song collections from 1990-2000, 2001-2010, and 2011-2018 each. The purpose is to see any sorts of emergent trends throughout the years, as well as to compare my listening habits to the genre in general.

In [4]:
# installing relevant library
%pip install spotipy


The following command must be run outside of the IPython shell:

    $ pip install spotipy

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [1]:
import pandas as pd
import spotipy
from spotipy import util

# Improve Pandas display settings
pd.set_option("display.width", 600)
pd.set_option("display.max_columns", 50)

# Spotify API limit
API_LIMIT = 50

In [2]:
# Authorizing my username and credentials with the spotipy library
username = 'jvichare'
scope = 'playlist-modify-public'
my_client_id = '02dd400e59cf44eb8d284cbcd27f10f3'
my_client_secret = '452063222fa84cec81cc2120017d9411'

spotify_token = util.prompt_for_user_token(username,
                                           scope,
                                           client_id=my_client_id,
                                           client_secret=my_client_secret,
                                           redirect_uri="http://127.0.0.1")

In [3]:
sp = spotipy.Spotify(auth=spotify_token)

In [84]:
#playlists_results = sp.user_playlists(username)
#playlist_name = "Metal"

#playlist_ids = [playlist['id'] for playlist in playlists_results['items'] if playlist['name'] == playlist_name]

#if not playlist_ids:
#    raise Exception("Cannot find playlist named: {}".format(playlist_name))
#else:
#    print(playlist_ids)

Exception: Cannot find playlist named: 'Metal'

In [31]:
#genres = sp.user_playlists('spotify')
#while genres:
#    for i, genre in enumerate(genres['items']):
#        print("%4d %s %s" % (i + 1 + genre['offset'], genre['uri'],  genre['name']))
#    if genre['next']:
#        genres = sp.next(genres)
#    else:
#        genres = None

KeyError: 'offset'

In [16]:
def _get_features_df(sp, track_ids):
    """
    This is an helper method to get track's features with pagination from track ids.
    It returns a Pandas dataframe
    """

    feature_list = []
    i = 0
    while track_ids:
        print("Call #{} for audio features".format(i + 1))
        features_results = sp.audio_features(track_ids[:API_LIMIT])

        feature_list += features_results
        
        track_ids = track_ids[API_LIMIT:]
        i += 1

    df_features = pd.DataFrame(feature_list)[["id", "analysis_url", "duration_ms", "acousticness", "danceability", 
                                              "energy", "instrumentalness", "liveness", "loudness", "valence", 
                                              "speechiness", "key", "mode", "tempo", "time_signature"]]
    # tempo is in range 24-200 ==> 0-176, normalize it
    df_features["tempo_norm"] = (df_features["tempo"] - 24) / 176.
    
    return df_features

In [17]:
def _get_artists_df(sp, artist_ids):
    """
    This is an helper method to get artist's information with pagination from artist ids.
    It returns a Pandas dataframe
    """

    artist_list = []
    i = 0

    while artist_ids:
        print("Call #{} for artists".format(i + 1))
        artists_results = sp.artists(artist_ids[:API_LIMIT])

        artist_list += [[t["id"], t["genres"], t["popularity"]] for t in artists_results["artists"]]

        artist_ids = artist_ids[API_LIMIT:]
        i += 1

    df_artists = pd.DataFrame(artist_list, columns=["artist_id", "artist_genres", "artist_popularity"])

    df_artists["artist_popularity_norm"] = df_artists["artist_popularity"] / 100.

    return df_artists

The process below was repeated for several genres by changing the genre tag. I could have probably added another for loop to do it all automated, but honestly I was lazy and worried I would query the Spotify servers too much otherwise.

The full list was `'progressive metal', 'jazz', 'classical', 'hip-hop', 'rap', 'pop', 'rock', 'indie rock', 'indie pop'`

In [72]:
number_of_tracks = 3000
genre = 'indie pop'

search_runs = int(number_of_tracks / API_LIMIT)

search_list = []
for i in range(search_runs):
    print("Call #{} for tracks".format(i+1))
    search_results = sp.search('genre:"{}"'.format(genre), type="track",  limit=API_LIMIT, offset=API_LIMIT*i)

    search_list += [[t["id"], t["name"], t["artists"][0]["id"], t["artists"][0]["name"],
                            t["album"]["name"], t["popularity"]]
                           for t in search_results['tracks']['items']]

df_search = pd.DataFrame(search_list, 
                         columns=["id", "song_name", "artist_id", "artist_name", "album_name", "popularity"])
df_search["popularity_norm"] = df_search["popularity"] / 100.
df_search.head()

Call #1 for tracks
Call #2 for tracks
Call #3 for tracks
Call #4 for tracks
Call #5 for tracks
Call #6 for tracks
Call #7 for tracks
Call #8 for tracks
Call #9 for tracks
Call #10 for tracks
Call #11 for tracks
Call #12 for tracks
Call #13 for tracks
Call #14 for tracks
Call #15 for tracks
Call #16 for tracks
Call #17 for tracks
Call #18 for tracks
Call #19 for tracks
Call #20 for tracks
Call #21 for tracks
Call #22 for tracks
Call #23 for tracks
Call #24 for tracks
Call #25 for tracks
Call #26 for tracks
Call #27 for tracks
Call #28 for tracks
Call #29 for tracks
Call #30 for tracks
Call #31 for tracks
Call #32 for tracks
Call #33 for tracks
Call #34 for tracks
Call #35 for tracks
Call #36 for tracks
Call #37 for tracks
Call #38 for tracks
Call #39 for tracks
Call #40 for tracks
Call #41 for tracks
Call #42 for tracks
Call #43 for tracks
Call #44 for tracks
Call #45 for tracks
Call #46 for tracks
Call #47 for tracks
Call #48 for tracks
Call #49 for tracks
Call #50 for tracks
Call #51 

,id,song_name,artist_id,artist_name,album_name,popularity,popularity_norm
0,4v0CuFW0bEMBQ1gRn2mB3X,Sit Next to Me - Stereotypes Remix,7gP3bB2nilZXLfPHJhMdvc,Foster The People,Sit Next to Me (Stereotypes Remix),83,0.83
1,0QZ5yyl6B6utIWkxeBDxQN,The Night We Met,6ltzsmQQbmdoHHbLZ4ZN25,Lord Huron,Strange Trails,78,0.78
2,4BdGO1CaObRD4La9l5Zanz,Sit Next to Me,7gP3bB2nilZXLfPHJhMdvc,Foster The People,Sacred Hearts Club,79,0.79
3,7w87IxuO7BDcJ3YUqCyMTT,Pumped Up Kicks,7gP3bB2nilZXLfPHJhMdvc,Foster The People,Torches,84,0.84
4,4kbj5MwxO1bq9wjT5g9HaA,Shut Up and Dance,6DIS6PRrLS3wbnZsf7vYic,WALK THE MOON,TALKING IS HARD,81,0.81


In [73]:
print(df_search['artist_name'].nunique())
df_search.shape

312


(3000, 7)

In [74]:
track_ids = df_search["id"].unique().tolist()
df_features = _get_features_df(sp, track_ids)
df_features.head()

Call #1 for audio features
Call #2 for audio features
Call #3 for audio features
Call #4 for audio features
Call #5 for audio features
Call #6 for audio features
Call #7 for audio features
Call #8 for audio features
Call #9 for audio features
Call #10 for audio features
Call #11 for audio features
Call #12 for audio features
Call #13 for audio features
Call #14 for audio features
Call #15 for audio features
Call #16 for audio features
Call #17 for audio features
Call #18 for audio features
Call #19 for audio features
Call #20 for audio features
Call #21 for audio features
Call #22 for audio features
Call #23 for audio features
Call #24 for audio features
Call #25 for audio features
Call #26 for audio features
Call #27 for audio features
Call #28 for audio features
Call #29 for audio features
Call #30 for audio features
Call #31 for audio features
Call #32 for audio features
Call #33 for audio features
Call #34 for audio features
Call #35 for audio features
Call #36 for audio features
C

,id,analysis_url,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,valence,speechiness,key,mode,tempo,time_signature,tempo_norm
0,4v0CuFW0bEMBQ1gRn2mB3X,https://api.spotify.com/v1/audio-analysis/4v0C...,190720,0.17300,0.680,0.738,0.000360,0.3700,-4.881,0.675,0.0331,6,0,105.015,4,0.460313
1,0QZ5yyl6B6utIWkxeBDxQN,https://api.spotify.com/v1/audio-analysis/0QZ5...,208227,0.96800,0.441,0.379,0.262000,0.6390,-9.545,0.105,0.0449,2,1,174.118,3,0.852943
2,4BdGO1CaObRD4La9l5Zanz,https://api.spotify.com/v1/audio-analysis/4BdG...,243147,0.11700,0.638,0.738,0.000384,0.6890,-4.976,0.657,0.0292,6,0,97.021,4,0.414892
3,7w87IxuO7BDcJ3YUqCyMTT,https://api.spotify.com/v1/audio-analysis/7w87...,239600,0.14500,0.733,0.710,0.115000,0.0956,-5.849,0.965,0.0292,5,0,127.975,4,0.590767
4,4kbj5MwxO1bq9wjT5g9HaA,https://api.spotify.com/v1/audio-analysis/4kbj...,199080,0.00701,0.578,0.866,0.000000,0.2570,-3.804,0.619,0.0619,1,1,128.038,4,0.591125


In [75]:
artist_ids = df_search["artist_id"].unique().tolist()
df_artists = _get_artists_df(sp, artist_ids)
df_artists.head()

Call #1 for artists
Call #2 for artists
Call #3 for artists
Call #4 for artists
Call #5 for artists
Call #6 for artists
Call #7 for artists


,artist_id,artist_genres,artist_popularity,artist_popularity_norm
0,7gP3bB2nilZXLfPHJhMdvc,"[indie pop, la indie, modern rock]",78,0.78
1,6ltzsmQQbmdoHHbLZ4ZN25,"[folk-pop, indie folk, indie pop, new american...",73,0.73
2,6DIS6PRrLS3wbnZsf7vYic,"[indie pop, indie poptimism, indietronica, mod...",74,0.74
3,5nCi3BB41mBaMH9gfr6Su0,"[indie pop, metropopolis, modern rock]",70,0.70
4,7giUHu5pv6YTZgSkxxCcgh,"[folk-pop, indie folk, indie pop, indie rock, ...",68,0.68


In [76]:
df_sample = df_features.merge(df_search, on="id")
df_sample = df_sample.merge(df_artists, on="artist_id")

df_sample["full_name"] = df_sample["artist_name"] + " -- " + df_sample["song_name"]
df_sample.sort_values("popularity", inplace=True, ascending=False)

df_sample.head()

,id,analysis_url,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,valence,speechiness,key,mode,tempo,time_signature,tempo_norm,song_name,artist_id,artist_name,album_name,popularity,popularity_norm,artist_genres,artist_popularity,artist_popularity_norm,full_name
2,7w87IxuO7BDcJ3YUqCyMTT,https://api.spotify.com/v1/audio-analysis/7w87...,239600,0.14500,0.733,0.710,0.115000,0.0956,-5.849,0.965,0.0292,5,0,127.975,4,0.590767,Pumped Up Kicks,7gP3bB2nilZXLfPHJhMdvc,Foster The People,Torches,84,0.84,"[indie pop, la indie, modern rock]",78,0.78,Foster The People -- Pumped Up Kicks
0,4v0CuFW0bEMBQ1gRn2mB3X,https://api.spotify.com/v1/audio-analysis/4v0C...,190720,0.17300,0.680,0.738,0.000360,0.3700,-4.881,0.675,0.0331,6,0,105.015,4,0.460313,Sit Next to Me - Stereotypes Remix,7gP3bB2nilZXLfPHJhMdvc,Foster The People,Sit Next to Me (Stereotypes Remix),83,0.83,"[indie pop, la indie, modern rock]",78,0.78,Foster The People -- Sit Next to Me - Stereoty...
86,4kbj5MwxO1bq9wjT5g9HaA,https://api.spotify.com/v1/audio-analysis/4kbj...,199080,0.00701,0.578,0.866,0.000000,0.2570,-3.804,0.619,0.0619,1,1,128.038,4,0.591125,Shut Up and Dance,6DIS6PRrLS3wbnZsf7vYic,WALK THE MOON,TALKING IS HARD,81,0.81,"[indie pop, indie poptimism, indietronica, mod...",74,0.74,WALK THE MOON -- Shut Up and Dance
1,4BdGO1CaObRD4La9l5Zanz,https://api.spotify.com/v1/audio-analysis/4BdG...,243147,0.11700,0.638,0.738,0.000384,0.6890,-4.976,0.657,0.0292,6,0,97.021,4,0.414892,Sit Next to Me,7gP3bB2nilZXLfPHJhMdvc,Foster The People,Sacred Hearts Club,79,0.79,"[indie pop, la indie, modern rock]",78,0.78,Foster The People -- Sit Next to Me
39,0QZ5yyl6B6utIWkxeBDxQN,https://api.spotify.com/v1/audio-analysis/0QZ5...,208227,0.96800,0.441,0.379,0.262000,0.6390,-9.545,0.105,0.0449,2,1,174.118,3,0.852943,The Night We Met,6ltzsmQQbmdoHHbLZ4ZN25,Lord Huron,Strange Trails,78,0.78,"[folk-pop, indie folk, indie pop, new american...",73,0.73,Lord Huron -- The Night We Met


In [77]:
# Convert time_signature and key to category
df_sample["time_signature"] = df_sample["time_signature"].astype(pd.api.types.CategoricalDtype(categories=[1, 2, 3, 4, 5]))
df_sample["key"] = df_sample["key"].astype(pd.api.types.CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]))

In [78]:
print(df_sample.info())

df_sample.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000 entries, 2 to 1300
Data columns (total 26 columns):
id                        3000 non-null object
analysis_url              3000 non-null object
duration_ms               3000 non-null int64
acousticness              3000 non-null float64
danceability              3000 non-null float64
energy                    3000 non-null float64
instrumentalness          3000 non-null float64
liveness                  3000 non-null float64
loudness                  3000 non-null float64
valence                   3000 non-null float64
speechiness               3000 non-null float64
key                       3000 non-null category
mode                      3000 non-null int64
tempo                     3000 non-null float64
time_signature            3000 non-null category
tempo_norm                3000 non-null float64
song_name                 3000 non-null object
artist_id                 3000 non-null object
artist_name               3000 non-

,duration_ms,acousticness,danceability,energy,instrumentalness,liveness,loudness,valence,speechiness,mode,tempo,tempo_norm,popularity,popularity_norm,artist_popularity,artist_popularity_norm
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,235502.727000,0.277642,0.540099,0.639286,0.101436,0.185477,-7.398749,0.459256,0.048729,0.748000,121.420425,0.553525,53.693333,0.536933,64.850667,0.648507
std,60543.252381,0.322691,0.131085,0.225092,0.223285,0.149435,3.318559,0.226110,0.037907,0.434234,27.993040,0.159051,6.921411,0.069214,6.925004,0.069250
min,32413.000000,0.000003,0.135000,0.016100,0.000000,0.021100,-25.111000,0.027700,0.022600,0.000000,37.981000,0.079438,0.000000,0.000000,33.000000,0.330000
25%,198600.000000,0.012400,0.457000,0.482000,0.000007,0.098300,-8.752500,0.282000,0.031500,0.000000,100.040000,0.432045,49.000000,0.490000,60.000000,0.600000
50%,226693.000000,0.118000,0.547000,0.690000,0.000753,0.123000,-6.632000,0.447000,0.037800,1.000000,119.965000,0.545256,53.000000,0.530000,66.000000,0.660000
75%,261379.750000,0.496750,0.632000,0.823000,0.044775,0.228250,-5.140750,0.628250,0.051025,1.000000,138.785750,0.652192,58.000000,0.580000,70.000000,0.700000
max,798693.000000,0.995000,0.884000,0.994000,0.986000,0.983000,-0.734000,0.978000,0.705000,1.000000,219.331000,1.109835,84.000000,0.840000,78.000000,0.780000


In [79]:
df_sample.to_csv('indiepop_sample_3000.csv', index = False)